In [91]:
#data preprocessing
import pandas as pd
#produces a prediction model in the form of an ensemble of weak prediction models, typically decision tree
import xgboost as xgb
#the outcome (dependent variable) has only a limited number of possible values. 
#Logistic Regression is used when response variable is categorical in nature.
from sklearn.linear_model import LogisticRegression
#A random forest is a meta estimator that fits a number of decision tree classifiers 
#on various sub-samples of the dataset and use averaging to improve the predictive 
#accuracy and control over-fitting.
from sklearn.ensemble import RandomForestClassifier
#a discriminative classifier formally defined by a separating hyperplane.
from sklearn.svm import SVC
#displayd data
from IPython.display import display
%matplotlib inline

In [92]:
# Read data and drop redundant column.
data = pd.read_csv('Data/final_processed_2.csv')
display(data.head())


# Total number of matches.
n_matches = data.shape[0]

# Calculate number of features. -1 because we are saving one as the target variable (win/lose/draw)
n_features = data.shape[1] - 1

# Calculate matches won by home team.
n_homewins = len(data[data.FTR == 'H'])

# Calculate win rate for home team.
win_rate = (float(n_homewins) / (n_matches)) * 100


# Print the results
print ("Total number of matches: {}".format(n_matches))
print ("Number of features: {}".format(n_features))
print ("Number of matches won by home team: {}".format(n_homewins))
print ("Win rate of home team: {:.2f}%".format(win_rate))

,div,key,team1,team2,date,FTR,team1_home_win,team1_home_draw,team1_away_win,team1_away_draw,...,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,home_team_overall,away_team_overall
0,702,Stoke_Hull_2010-04-03,4,7,2010-04-03,H,0.472222,0.277778,0.147059,0.323529,...,65,69,61,72,58,71,72,59,429,431
1,341,Middlesbrough_Man United_2009-05-02,9,19,2009-05-02,A,0.277778,0.444444,0.117647,0.117647,...,76,69,60,62,67,81,69,76,395,420
2,1514,Stoke_Bolton_2012-05-13,4,3,2012-05-13,D,0.447368,0.302632,0.171053,0.250000,...,56,68,70,59,58,73,61,63,429,467
3,812,Blackpool_Blackburn_2010-09-25,24,6,2010-09-25,A,0.000000,0.500000,0.500000,0.000000,...,76,64,73,68,81,57,69,65,459,501
4,1682,Newcastle_Man City_2012-12-15,20,16,2012-12-15,A,0.393939,0.318182,0.230769,0.276923,...,55,58,78,80,67,75,60,77,430,370


Total number of matches: 3040
Number of features: 115
Number of matches won by home team: 1390
Win rate of home team: 45.72%


In [93]:
# Separate into feature set and target variable
#FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)
X_all =data.drop(['div','FTR', 'key', 'date', 'WHH','WHD','WHA'], axis=1)  # df.columns is zero-based pd.Index
# X_all =X_all.drop(0, axis=1)  # df.columns is zero-based pd.Index 

y_all = data['FTR']

# homeTeams = data['team1']
# awayTeams = data['team2']
# teamIds ={}
# id=1
# for team in homeTeams:
#     if team not in teamIds:
#         teamIds[team] = id
#         id +=1
        
# for team in awayTeams:
#     if team not in teamIds:
#         teamIds[team] = id
#         id +=1
        
# for index,row in X_all.iterrows():
#     row['team1'] = teamIds[row['team1']]
#     row['team2'] = teamIds[row['team2']]
#     X_all.loc[index] = row
    
# print(teamIds)
display(X_all.head())

,team1,team2,team1_home_win,team1_home_draw,team1_away_win,team1_away_draw,team1_home_goals,team1_away_goals,team1_home_goals_conceded,team1_away_goals_conceded,...,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,home_team_overall,away_team_overall
0,4,7,0.472222,0.277778,0.147059,0.323529,1.250000,0.735294,0.944444,1.647059,...,65,69,61,72,58,71,72,59,429,431
1,9,19,0.277778,0.444444,0.117647,0.117647,0.888889,0.529412,1.055556,1.882353,...,76,69,60,62,67,81,69,76,395,420
2,4,3,0.447368,0.302632,0.171053,0.250000,1.342105,0.684211,0.973684,1.710526,...,56,68,70,59,58,73,61,63,429,467
3,24,6,0.000000,0.500000,0.500000,0.000000,1.500000,1.500000,2.000000,2.500000,...,76,64,73,68,81,57,69,65,459,501
4,20,16,0.393939,0.318182,0.230769,0.276923,1.590909,1.015385,1.287879,1.661538,...,55,58,78,80,67,75,60,77,430,370


In [98]:
def preprocess_features(X):
    ''' Preprocesses the football data and converts catagorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
                    
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

# Standardising the data.
from sklearn.preprocessing import scale
# X_all = X_all.reset_index()
# X_all = X_all[X_all.label=='desired_one']
#Center to the mean and component wise scale to unit variance.
cols = X_all.columns

for col in cols:
    X_all[col] = scale(X_all[col])
X_all = preprocess_features(X_all)
# X_all.ha

/home/shravan/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [99]:
##splitting
from sklearn.model_selection import train_test_split
# train_test_split(X, y, test_size=0.2, shuffle=False)
# Shuffle and split the dataset into training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
                                                    test_size = 0.2,
                                                    shuffle = False)

In [100]:
#for measuring training time
from time import time 
# F1 score (also F-score or F-measure) is a measure of a test's accuracy. 
#It considers both the precision p and the recall r of the test to compute 
#the score: p is the number of correct positive results divided by the number of 
#all positive results, and r is the number of correct positive results divided by 
#the number of positive results that should have been returned. The F1 score can be 
#interpreted as a weighted average of the precision and recall, where an F1 score 
#reaches its best value at 1 and worst at 0.
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    
    return 0,sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print (f1, acc)
    print ("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print ("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

In [ ]:
# Initialize the three models (XGBoost is initialized later)
clf_A = LogisticRegression(random_state = 42)
clf_B = SVC(random_state = 912, kernel='rbf')
#Boosting refers to this general problem of producing a very accurate prediction rule 
#by combining rough and moderately inaccurate rules-of-thumb
clf_C = xgb.XGBClassifier(seed = 82)

train_predict(clf_A, X_train, y_train, X_test, y_test)
print ('')

train_predict(clf_B, X_train, y_train, X_test, y_test)
print ('')
train_predict(clf_C, X_train, y_train, X_test, y_test)
print ('')



Training a LogisticRegression using a training set size of 2432. . .
